In [1]:
import csv
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg


lines = []

with open('learned_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)


        
lines_offroad = []        
        
with open('offroad_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line_offroad in reader:
        lines_offroad.append(line_offroad)        

        
lines_challenge = []

with open('challenge_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line_challenge in reader:
        lines_challenge.append(line_challenge)     
               
    
images = []
measurements = []


for line in lines:
    
    # Center Camera
    
    source_path = line[0]
    filename = source_path.split('\\')[-1]
    current_path = 'learned_data/IMG/' + filename
    image = mpimg.imread(current_path)
    image_flipped = np.fliplr(image)
    images.append(image)
    images.append(image_flipped)
    
    measurement_center = np.float64(line[3])
    measurement_center_flipped = -np.copy(measurement_center)
    
    
    measurements.append(measurement_center)
    measurements.append(measurement_center_flipped)
    
    # Correction
    
    correction = 0.09

    #Left Camera
    
    source_path = line[1]
    filename = source_path.split('\\')[-1]
    current_path = 'learned_data/IMG/' + filename
    image = mpimg.imread(current_path)
    image_flipped = np.fliplr(image)
    images.append(image)
    
    measurement_left = np.float64(line[3]) + correction
    measurements.append(measurement_left)
    
    #Right Camera
    
    source_path = line[2]
    filename = source_path.split('\\')[-1]
    current_path = 'learned_data/IMG/' + filename
    image = mpimg.imread(current_path)
    image_flipped = np.fliplr(image)
    images.append(image)
    
    measurement_right = np.float64(line[3]) - correction
    measurements.append(measurement_right)
    
for line_offroad in lines_offroad:    
    # Offroad
    
    source_path = line_offroad[0]
    filename = source_path.split('\\')[-1]
    current_path = 'offroad_data/IMG/' + filename
    image = mpimg.imread(current_path)
    image_flipped = np.fliplr(image)
    images.append(image)
    images.append(image_flipped)
    
    measurement_center = np.float64(line_offroad[3])
    measurement_center_flipped = -np.copy(measurement_center)
    
    
    measurements.append(measurement_center)
    measurements.append(measurement_center_flipped)    
     
X_train = np.array(images)
print(np.shape(X_train))  
plt.imshow(X_train[28])
y_train = np.array(measurements)

(20174, 160, 320, 3)


In [2]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Lambda, Dropout, Convolution2D, MaxPooling2D, Cropping2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))

model.add(Convolution2D(24,5,5,subsample=(2,2), activation="relu")) 
model.add(Convolution2D(36,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2), activation="relu")) 

model.add(Convolution2D(64,3,3, activation="relu"))
model.add(Convolution2D(64,3,3, activation="relu"))

#model.add(Convolution2D(12,5,5, activation="relu")) 
#model.add(MaxPooling2D())
#model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')

model.fit(X_train,y_train,validation_split=0.2, shuffle=True, nb_epoch=5)


model.save('model.h5')






Using TensorFlow backend.
C:\01_Projekte\01_SDC_E\Miniconda\envs\car-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Train on 16139 samples, validate on 4035 samples
Epoch 1/5
16139/16139 [==============================] - 41s - loss: 0.0149 - val_loss: 0.1299
Epoch 2/5
16139/16139 [==============================] - 38s - loss: 0.0130 - val_loss: 0.1291
Epoch 3/5
16139/16139 [==============================] - 38s - loss: 0.0121 - val_loss: 0.1308
Epoch 4/5
16139/16139 [==============================] - 38s - loss: 0.0114 - val_loss: 0.1263
Epoch 5/5
16139/16139 [==============================] - 38s - loss: 0.0108 - val_loss: 0.1271
